In [1]:
import numpy as np

## Create a simple tensor with random items

In [8]:
params = np.random.uniform(-50,150,size=10000)

## Introduce an outlier
params[-1] = 1000
params     = np.round(params,2)

## Define the Quantization method and Quantize

In [12]:
def clamp(params_q:np.array,lower_bound:int,upper_bound:int) -> np.array:
    
    params_q[params_q<lower_bound] = lower_bound
    params_q[params_q>upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params:np.array,bits:int) -> tuple[np.array,float,int]:
    alpha  = params.max()
    beta   = params.min()
    scale  = (alpha- beta) / (2**bits - 1)
    zero   = -1 * np.round(beta / scale)
    lower_bound,upper_bound = 0, 2**bits - 1
    
    # Quantize the parameters
    quantize = clamp(np.round(params / scale + zero),lower_bound,upper_bound).astype(np.int32)
    
    return quantize,scale,zero


def asymmetric_quantization_percentile(params: np.array, bits: int, percentile: float = 99.99) -> tuple[np.array, float, int]:

    alpha = np.percentile(params, percentile)
    beta  = np.percentile(params, 100-percentile)
    scale = (alpha - beta) / (2**bits-1)
    zero  = -1*np.round(beta / scale)
    
    lower_bound, upper_bound = 0, 2**bits-1
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero
 
def asymmetric_dequantize(params_q:np.array, scale:float, zero:int) -> np.array:
    return (params_q - zero) * scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)


In [13]:
(asymmetric_q, asymmetric_scale, asymmetric_zero)                                  = asymmetric_quantization(params, 8)
(asymmetric_q_percentile, asymmetric_scale_percentile, asymmetric_zero_percentile) = asymmetric_quantization_percentile(params, 8)

In [14]:
print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric (min-max) scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print(f'')
print(f'Asymmetric (percentile) scale: {asymmetric_scale_percentile}, zero: {asymmetric_zero_percentile}')
print(asymmetric_q_percentile)

Original:
[  97.93  -40.98   95.21 ...   25.41  118.46 1000.  ]

Asymmetric (min-max) scale: 4.117529411764706, zero: 12.0
[ 36   2  35 ...  18  41 255]

Asymmetric (percentile) scale: 0.7844901999976426, zero: 64.0
[189  12 185 ...  96 215 255]


In [15]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_asymmetric_percentile = asymmetric_dequantize(asymmetric_q_percentile, asymmetric_scale_percentile, asymmetric_zero_percentile)


In [16]:
print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantized (min-max):')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantized (percentile):')
print(np.round(params_deq_asymmetric_percentile,2))

Original:
[  97.93  -40.98   95.21 ...   25.41  118.46 1000.  ]

Dequantized (min-max):
[  98.82  -41.18   94.7  ...   24.71  119.41 1000.56]

Dequantized (percentile):
[ 98.06 -40.79  94.92 ...  25.1  118.46 149.84]


## Evaluate the quantization error (excluding the outlier)

In [33]:
# Calculate the quantization error
print(f'{"Error (min-max) excluding outlier: ":>40}{np.round(quantization_error(params[:-1], params_deq_asymmetric[:-1]),2)}')
print(f'{"Error (percentile) excluding outlier: ":>40}{np.round(quantization_error(params[:-1], params_deq_asymmetric_percentile[:-1]), 2)}')


     Error (min-max) excluding outlier: 1.4
  Error (percentile) excluding outlier: 0.05
